# SPOTIFY API DATA MINING

### Introduction

lorem impsum lorem impsum lorem impsum lorem impsum lorem impsum 

### Objective

lorem impsum lorem impsum lorem impsum lorem impsum lorem impsum 

### Conclusion

lorem impsum lorem impsum lorem impsum lorem impsum lorem impsum 

---

## INDEX

### [Data Techniques](#DATA-TECHNIQUES)<br>

### [Code structure:](#CODE-STRUCTURE)
-  [Library imports](#LIBRARY-imports)
-  [1 - Authentication - API Auth Methods](#1-|-AUTHENTICATION)
-  [2 - Data Collection - API Get Methods](#2-|-DATA-COLLECTION)
-  [3 - Data Processing - DarkMagic](#3-|-DATA-PROCESSING)
-  [4 - Data Visualization - DataViz](#4-|-DATA-VISUALIZATION)

### [Analysis Methods](#ANALYSIS-Methods)


---

## DATA TECHNIQUES
[>>> back to the INDEX](#INDEX)

##### 1. Data Cleaning 
##### 2. Association
##### 3. Clustering
##### 4. Classification 
##### 5. Data Visualization
##### 6. Machine Learning 
##### 7. Prediction
##### 8. Neural Networks
##### 9. Outlier Detection
##### Data Warehousing

---

---

## CODE STRUCTURE
- [LIBRARY imports](#1-|-LIBRARY-imports)
- [1 | AUTHENTICATION - API Auth Methods](#1-|-AUTHENTICATION)
- [2 | DATA COLLECTION - API Get Methods](#2-|-DATA-COLLECTION)
- [3 | DATA PROCESSING - DarkMagic](#3-|-DATA-PROCESSING)
- [4 | DATA VISUALIZATION - DataViz](#4-|-DATA-VISUALIZATION)
<br><br>[>>>  back to the INDEX](#INDEX)

---

### LIBRARY imports
[>>> back to the INDEX](#INDEX)

In [1]:
### --- Libraries I am currently using:
# Environment
import sys
    
# Data Collection
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
# import tekore as tk - spotipy alternative

# Data Manipulation
import pandas as pd
import json
import time 

# Oth
import argparse
import logging

### --- Libraries I still want to use:
# DataViz
import matplotlib.pyplot as plt
# import ggplot as ggp
import seaborn as sns

# ML and DL
# import PyTorch
# import Caffe2
# import TensorFlow
# import Keras
# import Theano

ModuleNotFoundError: No module named 'spotipy'

---

### 1 | AUTHENTICATION
API Auth Methods<br><br>
[>>> back to the INDEX](#INDEX)

In [ ]:
cid="7ff0bc048c5b4f668d0bf5e109b689c9"
cs="59891a247ab54d609e45ca4f7253e86d"

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=cs)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

---

### 2 | DATA COLLECTION
API Get Methods
- [Get artist info](#Get-artist-info)
- Get Albums
- [Get tracks](#Get-tracks)
- Get tracks info
<br><br>[>>> back to the INDEX](#INDEX)

#### Get artist info

In [ ]:
def get_artist(artist):
    results = sp.search(q='artist:' + artist, type='artist')
    items = results['artists']['items']
    
    if len(items) > 0:
        if  pd.Series(items[0])[8] == 'artist':
            return pd.Series(items[0])
        else:
            return print('get_artist: This search retrieves item not artist')
    else:
        return None

# ### EXAMPLE
# get_artist(artist=artist_name)

#### Get Albums

In [ ]:
# from ARTIST
def get_albums_from_artist(artist):
    albums = []
    list_of_albums=[]
    results = sp.artist_albums(artist['id'], album_type='album')
    albums = results['items']
    first_artist_of_album = results['items'][0]['artists'][0]['name'] #
    
    for i in range (0,len(albums)):
        album_name = results['items'][i]['name']
        list_of_albums.append(album_name)
            
    return list_of_albums

#### Get Tracks

In [ ]:
# GET TRACKS FROM ARTIST NAME
track_list = []

def get_artist(name):
    results = sp.search(q='artist:' + name, type='artist')
    items = results['artists']['items']
    if len(items) > 0:
        return items[0]
    else:
        return None

def show_album_tracks(album):
    tracks = []
    results = sp.album_tracks(album['id'])
    tracks = results['items']    
    first_artist_of_album = results['items'][0]['artists'][0]['name'] #
    for i in range(0,len(tracks)):
#         if first_artist_of_album == artist_name: ############### FILTERING albums that are not primarily by the artist
        track_id = results['items'][i]['id']
        track_list.append(track_id)
    return track_list

def show_artist_albums(artist):
    albums = []
    results = sp.artist_albums(artist['id'], album_type='album')
    albums.extend(results['items'])
    while results['next']:
        results = sp.next(results)
        albums.extend(results['items'])
    unique = set()  # skip duplicate albums
    for album in albums:
        name = album['name'].lower()
        if name not in unique:
            unique.add(name)
            track_list=show_album_tracks(album)            
    return track_list

def get_tracks(name):
    artist = get_artist(name)
    track_list=show_artist_albums(artist)
    return track_list


#### Get track info

In [ ]:
# shows track info for a URN or URL
def get_track_info(track_id):
    track = sp.track('spotify:track:'+track_id)
    return track

In [ ]:
# Get all artists from playlist
def get_artist_of_track(track):
    artist_of_track = sp.track('spotify:track:'+track)['album']['artists'][0]['name']
    return artist_of_track

def get_artists_from_playlist(playlist):
    artists_from_playlist=[]
    response = sp.playlist_items(str('spotify:playlist:'+playlist),
                                 offset=0,
                                 fields='items.track.id,total',
                                 additional_types=['track'])
    for i in range (0, len(response['items'])):
        track_id = (str(response['items'][i]['track']['id']))
        artist_id = get_artist_of_track(track_id)
        artists_from_playlist.append(artist_id)
    return [*set(artists_from_playlist)]
# EXAMPLE
# artists_from_playlist=get_artists_from_playlist('0TRh1AAvKvpWd7JpMmDKMQ')
# print(artists_from_playlist)

In [ ]:
def get_discography_from_all_arists_from_playlists(list_of_playlists):
    all_discography = pd.DataFrame()
    for p in list_of_playlists:
#         print(list_of_playlists)
        artists=get_artists_from_playlist(p)
#         print(artists)
        for a in artists:
#             print(a)
            tracks=pd.DataFrame(get_tracks(a))
#             print(tracks)
            all_discography = pd.concat([all_discography, tracks], ignore_index=True).drop_duplicates()
    return all_discography

# EXAMPLE
# print(get_discography_from_all_arists_from_playlists(['0TRh1AAvKvpWd7JpMmDKMQ']))

---

### 3 | DATA PROCESSING
- [1 - Initial Table Modeling](#3.1-|-Initial-Table-Modeling)
- [2 - ODS Modeling](#3.2-|-ODS-Modeling)
<br><br>[>>> back to the INDEX](#INDEX)

#### 3.1 | Feature gathering & enrinchment

In [ ]:
def get_artist_features(artist):
# Getting artist basic data:
    artist_id = get_artist(artist)[4] 
    name = get_artist(artist)[6] 
    genres = get_artist(artist)[2] # str(get_artist(artist)[2]).replace("[","").replace("]","")
    popularity = get_artist(artist)[7] 
    followers = get_artist(artist)[1]['total']
    
    #Getting other artist data
    #Setting output
    artist_features = [artist_id, name, genres, popularity, followers]
    return artist_features

### EXAMPLE
# get_artist_features(artist='Astrix') 

In [ ]:
playlist = '0TRh1AAvKvpWd7JpMmDKMQ'
track_list = get_discography_from_all_arists_from_playlists([playlist])
print(track_list)

In [ ]:
# INFO ENDPOINT
# shows info for given tracks
tracks=pd.DataFrame(track_list).reset_index()
df_all_tracks_info = pd.DataFrame()

for i in range(0, len(tracks)):
    try:
        get_track_info = sp.track(track_id=tracks[0][i])
        data = {
            'track_id':get_track_info['id'],
            'track_name':get_track_info['name'],
            'track_popularity':get_track_info['popularity'],
            'track_number_in_album':get_track_info['track_number'],
            'track_duration_ms':get_track_info['duration_ms'],
            'artist_id':get_track_info['artists'][0]['id'],
            'artist_name':get_track_info['artists'][0]['name']
        }
        df_track_info = pd.DataFrame(data, index=[0])
        df_all_tracks_info = pd.concat([df_all_tracks_info,df_track_info], axis=0, ignore_index=True)
        df_all_tracks_info = df_all_tracks_info.drop_duplicates()
    except KeyError:
        continue
df_all_tracks_info.head()

In [ ]:
# FEATURES ENDPOINT
# shows acoustic features for given tracks

tracks=track_list[0]
df_all_tracks_features = pd.DataFrame()
for i in range(0,len(tracks)):
    try:
        get_track_features = sp.audio_features(tracks[i])[0]
        data = {
            'danceability': get_track_features['danceability'],
            'energy': get_track_features['energy'],
            'key': get_track_features['key'],
            'loudness': get_track_features['loudness'],
            'mode': get_track_features['mode'],
            'speechiness': get_track_features['speechiness'],
            'acousticness': get_track_features['acousticness'],
            'instrumentalness': get_track_features['instrumentalness'],
            'valence': get_track_features['valence'],
            'tempo': get_track_features['tempo'],
            'time_signature': get_track_features['time_signature']
        }
        df_in_analysis_track_feature = pd.DataFrame(data, index=[0])
        df_all_tracks_features = pd.concat([df_all_tracks_features, df_in_analysis_track_feature], axis=0)
    except KeyError:
        continue
df_all_tracks_features.head()


In [ ]:
# ANALYSIS ENDPOINT
# shows analysis for given tracks
tracks=track_list[0]
df_all_tracks_analysis = pd.DataFrame()

for i in range(0, len(tracks)):
    try:
        get_track_analysis = sp.audio_analysis(track_id=tracks[i])['track']
        data = {       
            'duration_s':get_track_analysis['duration'], 
            'loudness':get_track_analysis['loudness'],
            'analysis_sample_rate':get_track_analysis['analysis_sample_rate'],
            'tempo':get_track_analysis['tempo'],
            'time_signature':get_track_analysis['time_signature'],
            'key':get_track_analysis['key'],
            'key_confidence':get_track_analysis['key_confidence'],
            'mode':get_track_analysis['mode'], 
            'mode_confidence':get_track_analysis['mode_confidence'],
            'codestring':get_track_analysis['codestring'],
            'echoprintstring':get_track_analysis['echoprintstring'],
            'synchstring':get_track_analysis['synchstring'],
            'rhythmstring':get_track_analysis['rhythmstring']
        }
        df_track_analysis = pd.DataFrame(data, index=[0])
        df_all_tracks_analysis = pd.concat([df_all_tracks_analysis,df_track_analysis], axis=0)
    except KeyError:
        continue
df_all_tracks_analysis.head()


#### 3.2 | ODS Modeling

In [ ]:
print('ALL DATA FOR TRACKS:')
df_all_tracks_data = pd.concat([df_all_tracks_info, df_all_tracks_features, df_all_tracks_analysis], axis=1)
df_all_tracks_data = df_all_tracks_data.loc[:,~df_all_tracks_data.columns.duplicated()].copy()
df_all_tracks_data = df_all_tracks_data.drop_duplicates()
df_all_tracks_data.info()
print("-------------------------------")
print('ALL NUMERICA DATA FOR TRACKS:')
df_track_data_numerical = df_all_tracks_data.drop(columns=['track_id','track_name','artist_id','artist_name','codestring','echoprintstring','synchstring','rhythmstring','analysis_sample_rate','duration_s'])#'artist_id','artist_name','codestring','echoprintstring','synchstring','rythmstring'])
df_track_data_numerical = df_track_data_numerical.drop_duplicates()
df_track_data_numerical = df_track_data_numerical.reset_index()
df_track_data_numerical.info()

---

### 4 | DATA VISUALIZATION
[>>> back to the INDEX](#INDEX)

In [ ]:
# Histogram of all numerical
df_track_data_numerical.hist(figsize=(16, 20), bins=30, xlabelsize=8, ylabelsize=8)

In [ ]:
# Calculate Correlations
df_num_corr = df_track_data_numerical.corr()

# Print all correlation with popularity sorted by most correlated to least
print(df_num_corr['track_popularity'].sort_values(ascending=False)) 

In [ ]:
# Seaborn - Pairplot based on Y column
for i in range(0, len(df_track_data_numerical.columns), 5):
    sns.pairplot(data=df_track_data_numerical,
                x_vars=df_track_data_numerical.columns[i:i+5],
                y_vars=['track_popularity'])

In [ ]:
# Seaborn - Pairplot (ALL)
sns.set_theme(style="ticks")
sns.pairplot(df_track_data_numerical) #hue="species"

In [ ]:
# Single column describe + plot data distriubtion
print(df_track_data_numerical['track_popularity'].describe())
plt.figure(figsize=(9, 8))
sns.distplot(df_track_data_numerical['track_popularity'], color='g', bins=100, hist_kws={'alpha': 0.4});

In [ ]:
# Correlation map
corr = df_track_data_numerical.drop('track_popularity', axis=1).corr() # We already examined SalePrice correlations
plt.figure(figsize=(12, 10))

sns.heatmap(corr[(corr >= 0.1) | (corr <= -0.1)], 
            cmap='viridis', vmax=1.0, vmin=-1.0, linewidths=0.1,
            annot=True, annot_kws={"size": 8}, square=True);

---

## ANALYSIS Methods
Track Audio Analysis


<br><br>[>>> back to the INDEX](#INDEX)

Data Hierarchy
Artist > Album > Track >